# Smith-Waterman local alignment algorithm

Load modules required for the implementation.

In [1]:
import numpy as np
from operator import itemgetter

For a high-level description of the Smith-Waterman algorithm, see, e.g., the [Wikipedia entry](https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm).

Define two sequences to align.

In [2]:
seq1 = 'AGCACACA'
seq2 = 'ACACACTA'

Define similarity function.

In [3]:
def similarity(a, b):
    return 2 if a == b else -1

Define gap scoring.

In [4]:
gap_scoring = -1

Define function to initalize the scoring matrix.

In [5]:
def init_scoring_matrix(m, n):
    matrix = np.empty((m + 1, n + 1), dtype=int, order='C')
    matrix[0, :] = 0
    matrix[:, 0] = 0
    return matrix

Define the function that initializes the data structure to hold the path through the scoring matrix. This will be a matrix of tuples, where each tuple denotes a position in the scoring matrix, i.e., the one it was derived from. Note that this  could be implemented as an `(m, n, 2)` matrix as well.

In [6]:
def init_path_matrix(m, n):
    return [[(-1, -1) for j in range(n)] for i in range(m)]

Define the function to compute the score at a given matrix position.

In [7]:
def compute_score(seq1, seq2, matrix, i, j):
    upper_left = (matrix[i - 1, j - 1] + similarity(seq1[i - 1], seq2[j - 1]), (i - 1, j - 1))
    upper = (max([matrix[i -  k, j] + gap_scoring for k in range(1, i + 1)]), (i - 1, j))
    left = (max([matrix[i, j - l] + gap_scoring for l in range(1, j + 1)]), (i, j - 1))
    return max([(0, (i - 1, j - 1)), upper_left, upper, left], key=itemgetter(0))

Function to compute the scoring matrix for two given sequences.

In [8]:
def compute_scoring(seq1, seq2):
    scoring_matrix = init_scoring_matrix(len(seq1), len(seq2))
    path_matrix = init_path_matrix(len(seq1), len(seq2))
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            scoring_matrix[i, j], path_matrix[i - 1][j - 1] = compute_score(seq1, seq2, scoring_matrix, i, j)
    return scoring_matrix, path_matrix

Compute the scoring and the path matrices for the two given sequences.

In [9]:
scoring_matrix, path_matrix = compute_scoring(seq1, seq2)

In [34]:
scoring_matrix

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  2,  1,  2,  1,  2,  1,  1,  2],
       [ 0,  1,  1,  1,  1,  1,  1,  0,  1],
       [ 0,  1,  3,  2,  3,  2,  3,  2,  2],
       [ 0,  2,  2,  5,  4,  5,  4,  4,  4],
       [ 0,  1,  4,  4,  7,  6,  7,  6,  6],
       [ 0,  2,  3,  6,  6,  9,  8,  8,  8],
       [ 0,  1,  4,  5,  8,  8, 11, 10, 10],
       [ 0,  2,  3,  6,  7, 10, 10, 10, 12]])

In [10]:
path_matrix

[[(0, 0), (1, 1), (0, 2), (1, 3), (0, 4), (1, 5), (1, 6), (0, 7)],
 [(1, 1), (1, 1), (1, 3), (1, 3), (1, 5), (1, 5), (1, 6), (1, 8)],
 [(2, 1), (2, 1), (3, 2), (2, 3), (3, 4), (2, 5), (3, 6), (3, 7)],
 [(3, 0), (3, 2), (3, 2), (4, 3), (3, 4), (4, 5), (4, 6), (3, 7)],
 [(4, 1), (4, 1), (4, 3), (4, 3), (5, 4), (4, 5), (5, 6), (5, 7)],
 [(5, 0), (5, 2), (5, 2), (5, 4), (5, 4), (6, 5), (6, 6), (5, 7)],
 [(6, 1), (6, 1), (6, 3), (6, 3), (6, 5), (6, 5), (7, 6), (7, 7)],
 [(7, 0), (7, 2), (7, 2), (7, 4), (7, 4), (7, 6), (7, 6), (7, 7)]]

Define a function that computes the actual alignment based on the scoring and the path matrix.

In [28]:
def compute_alignment(seq1, seq2, scoring_matrix, path_matrix):
    a_seq1, a_seq2 = '', ''
    max_pos = np.unravel_index(np.argmax(scoring_matrix), scoring_matrix.shape, order='C')
    x, y = max_pos[0] - 1, max_pos[1] - 1
    while (scoring_matrix[x + 1, y + 1] != 0):
        new_x = path_matrix[x][y][0] - 1
        new_y = path_matrix[x][y][1] - 1
        if x == new_x:
            a_seq1 = '-' + a_seq1
            a_seq2 = seq2[y] + a_seq2
        elif y == new_y:
            a_seq1 = seq1[x] + a_seq1
            a_seq2 = '-' + a_seq2
        else:
            a_seq1 = seq1[x] + a_seq1
            a_seq2 = seq2[y] + a_seq2
        x, y = new_x, new_y
    return a_seq1, a_seq2

Compute the alignment for the example sequences, and print them aligned.

In [29]:
print('\n'.join(compute_alignment(seq1, seq2, scoring_matrix, path_matrix)))

AGCACAC-A
A-CACACTA


Define a function that ties all together.

In [30]:
def align(seq1, seq2):
    scoring_matrix, path_matrix = compute_scoring(seq1, seq2)
    return compute_alignment(seq1, seq2, scoring_matrix, path_matrix)